In [13]:
%pip install yfinance pandas "numpy<2" scikit-learn matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


In [14]:
#import libraries

import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [15]:
#download BITB data 
ticker = "BITB"
df = yf.Ticker(ticker).history(period="max")
df = df[['Open','High','Low','Close','Volume']].copy()


In [16]:
#compute daily returns
df['return'] = df['Close'].pct_change() 
df = df.dropna()

In [17]:
#Feature Engineering (process of creating, modifying, or selecting input variables in a dataset)
#this cell will create lagged returns(?), moving averages(?), volatility, and volume ratio

df['r_lag1'] = df['return'].shift(1)

df['ma5'] = df['return'].rolling(window=5).mean().shift(1)
df['ma10'] = df['return'].rolling(window=10).mean().shift(1)

# volatility (rolling std)
df['vol5'] = df['return'].rolling(window=5).std().shift(1)
df['vol10'] = df['return'].rolling(window=10).std().shift(1)

# volume relative to 10-day average
df['vol_avg10'] = df['Volume'].rolling(window=10).mean().shift(1)
#takes the average trading colyne over the past 10 days 
df['vol_ratio'] = df['Volume'] / df['vol_avg10']

# Drop rows with NaNs produced by shifting/rolling
df = df.dropna()

In [18]:
#Train-Test Split (time series) 

train_size = int(len(df) * 0.8)

train_df = df.iloc[:train_size]

test_df = df.iloc[train_size:]

In [19]:
#Scale Feautures
from sklearn.preprocessing import StandardScaler

features = ["r_lag1", "ma5", "vol10", "vol_ratio"]
scaler = StandardScaler()

X_train = scaler.fit_transform(train_df[features])
X_test = scaler.transform(test_df[features])

y_train = train_df["return"].values
y_test = test_df["return"].values


In [20]:
#Fitting the Linear Regression model
model = LinearRegression()
model = RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0])
model.fit(X_train, y_train)

,"alphas alphas: array-like of shape (n_alphas,), default=(0.1, 1.0, 10.0)Array of alpha values to try.Regularization strength; must be a positive float. Regularizationimproves the conditioning of the problem and reduces the variance ofthe estimates. Larger values specify stronger regularization.Alpha corresponds to ``1 / (2C)`` in other linear models such as:class:`~sklearn.linear_model.LogisticRegression` or:class:`~sklearn.svm.LinearSVC`.If using Leave-One-Out cross-validation, alphas must be strictly positive.","[0.01, 0.1, ...]"
,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto false, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"scoring scoring: str, callable, default=NoneThe scoring method to use for cross-validation. Options:- str: see :ref:`scoring_string_names` for options.- callable: a scorer callable object (e.g., function) with signature ``scorer(estimator, X, y)``. See :ref:`scoring_callable` for details.- `None`: negative :ref:`mean squared error ` if cv is None (i.e. when using leave-one-out cross-validation), or :ref:`coefficient of determination ` (:math:`R^2`) otherwise.",None
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the efficient Leave-One-Out cross-validation- integer, to specify the number of folds.- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if ``y`` is binary or multiclass,:class:`~sklearn.model_selection.StratifiedKFold` is used, else,:class:`~sklearn.model_selection.KFold` is used.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here.",None
,"gcv_mode gcv_mode: {'auto', 'svd', 'eigen'}, default='auto'Flag indicating which strategy to use when performingLeave-One-Out Cross-Validation. Options are:: 'auto' : use 'svd' if n_samples > n_features, otherwise use 'eigen' 'svd' : force use of singular value decomposition of X when X is dense, eigenvalue decomposition of X^T.X when X is sparse. 'eigen' : force computation via eigendecomposition of X.X^TThe 'auto' mode is the default and is intended to pick the cheaperoption of the two depending on the shape of the training data.",None
,"store_cv_results store_cv_results: bool, default=FalseFlag indicating if the cross-validation values corresponding toeach alpha should be stored in the ``cv_results_`` attribute (seebelow). This flag is only compatible with ``cv=None`` (i.e. usingLeave-One-Out Cross-Validation)... versionchanged:: 1.5 Parameter name changed from `store_cv_values` to `store_cv_results`.",False
,"alpha_per_target alpha_per_target: bool, default=FalseFlag indicating whether to optimize the alpha value (picked from the`alphas` parameter list) for each target separately (for multi-outputsettings: multiple prediction targets). When set to `True`, afterfitting, the `alpha_` attribute will contain a value for each target.When set to `False`, a single alpha is used for all targets... versionadded:: 0.24",False


In [ ]:
joblib.dump(model, "model.pkl")
joblib.dump(scaler, "scaler.pkl")

['model.pkl']